In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math as m
import mahotas as mt
import os
import pandas as pd
cv2.__version__

'4.0.1'

In [2]:
path = "C:/Users/suhas/Documents/Mini Project/Leaves/"

## Create Columns and DataFrame

In [3]:
columns = ['common name','red_mean', 'red_var', 'blue_mean', 'blue_var', 'green_mean', 'green_var', 'area', 'perimeter', 'length', 'width', 'aspect ratio', 'form factor', 'rectangularity', 'perimeter ratio of length and width', 'contrast', 'correlation', 'entropy', 'inverse difference moments']

In [4]:
columns

['common name',
 'red_mean',
 'red_var',
 'blue_mean',
 'blue_var',
 'green_mean',
 'green_var',
 'area',
 'perimeter',
 'length',
 'width',
 'aspect ratio',
 'form factor',
 'rectangularity',
 'perimeter ratio of length and width',
 'contrast',
 'correlation',
 'entropy',
 'inverse difference moments']

In [5]:
df = pd.DataFrame(columns=columns)

In [6]:
df

,common name,red_mean,red_var,blue_mean,blue_var,green_mean,green_var,area,perimeter,length,width,aspect ratio,form factor,rectangularity,perimeter ratio of length and width,contrast,correlation,entropy,inverse difference moments


## Create Label Function

In [7]:
def label(img_name):
    num = img_name.split('_')[1]
    #print(num)
    if num == 'pubescent bamboo':
        return 0 
    elif num == 'chinese horse chestnut':
        return 1 
    elif num == 'anhui barberry':
        return 2 
    elif num == 'chinese redbud':
        return 3
    elif num == 'true indigo':
        return 4
    elif num == 'japanese maple':
        return 5 
    elif num == 'nanmu':
        return 6 
    elif num == 'castor aralia':
        return 7 
    elif num == 'chinese cinnamon':
        return 8 
    elif num == 'goldenrain tree':
        return 9 
    elif num == 'big fruited holly':
        return 10 
    elif num == 'japanese chestwood':
        return 11 
    elif num == 'wintersweet':
        return 12 
    elif num == 'camphor tree':
        return 13
    elif num == 'japan arrowwood':
        return 14
    elif num == 'sweet osmanthus':
        return 15
    elif num == 'deodar':
        return 16
    elif num == 'ginkgo':
        return 17 
    elif num == 'crape myrtle':
        return 18 
    elif num == 'oleandar':
        return 19
    elif num == 'yew plum pie':
        return 20
    elif num == 'japanese flowering cherry':
        return 21 
    elif num == 'glossy privet':
        return 22
    elif num == 'chinese toon':
        return 23
    elif num == 'peach':
        return 24
    elif num == 'ford woodlotus':
        return 25
    elif num == 'trident maple':
        return 26
    elif num == 'beales barberry':
        return 27
    elif num == 'southern magnolia':
        return 28
    elif num == 'canadian poplar':
        return 29
    elif num == 'chinese tulip tree':
        return 30
    elif num == 'tangerine':
        return 31
    elif num == 'ulmus carpinifolia':
        return 32 
    elif num == 'acer':
        return 33
    elif num == 'salix aurita':
        return 34
    elif num == 'quercus':
        return 35 
    elif num == 'alnus incana':
        return 36 
    elif num == 'betula pubescens':
        return 37 
    elif num == 'salix alba sericea':
        return 38 
    elif num == 'populus tremula':
        return 39 
    elif num == 'ulmus glabra':
        return 40 
    elif num == 'sorbus aucuparia':
        return 41 
    elif num == 'salix sinerea':
        return 42 
    elif num == 'populus':
        return 43
    elif num == 'tilia':
        return 44
    elif num == 'sorbus intermedia':
        return 45
    elif num == 'fagus silvatica':
        return 46

## Iterating Over Images and extracting features

In [8]:
index = 0
for folder in os.scandir(path):
    print(folder)
    if "leaf" in str(folder):
        for image in os.scandir(os.path.join(path, folder)):
            # Getting Name of Plant
            name = folder.name.split('_')[1]
            #print(name)
            # Getting img
            img = cv2.imread(os.path.join(path, folder, image))

            # Getting Colour Based Features
            # Red
            red = img.copy()
            red[:, :, 0] = 0
            red[:, :, 1] = 0
            red_mean = np.mean(red[:, :, 2])
            red_var = np.std(red[:, :, 2])

            # Blue
            blue = img.copy()
            blue[:, :, 1] = 0
            blue[:, :, 2] = 0 
            blue_mean = np.mean(blue[:, :, 0])
            blue_var = np.std(blue[:, :, 0])

            # Green
            green = img.copy()
            green[:, :, 0] = 0
            green[:, :, 2] = 0
            green_mean = np.mean(green[:, :, 1])
            green_var = np.std(green[:, :, 1])

            # Pre-processing for remaining features
            imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 0)
            thresh, imgBW = cv2.threshold(imgBlur, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
            imgInv = cv2.bitwise_not(imgBW)
            kernel = np.ones((50, 50))
            imgClosed = cv2.morphologyEx(imgInv, cv2.MORPH_CLOSE, kernel)
            contours, _ = cv2.findContours(imgClosed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            l = 0
            for i in contours:
                if len(i) >l:
                    l = len(i)
                    cnt = i

            # Shape Based Features
            area = cv2.contourArea(cnt)
            per = cv2.arcLength(cnt, True)
            rect = cv2.minAreaRect(cnt)
            (x, y), (w, h), angle = rect

            # Morphological Features
            ar = float(h) / w
            ff = (4 * m.pi * area) / (m.pow(per, 2))
            rectangularity = (h * w) / area
            #nf = dia / h
            #pr = per / dia
            prlw = per / (w + h)

            # Texture Based Features
            textures = mt.features.haralick(imgGray)
            textures  = textures.mean(axis=0)
            contrast = textures[1]
            correlation = textures[2]
            entropy = textures[8]
            idf = textures[4]

            df = df.append({'common name': name, 'red_mean': red_mean, 'red_var': red_var, 'blue_mean': blue_mean, 'blue_var': blue_var, 'green_mean': green_mean, 'green_var': green_var, 'area': area, 'perimeter': per, 'length': h, 'width': w, 'aspect ratio': ar, 'form factor': ff, 'rectangularity':rectangularity, 'perimeter ratio of length and width': prlw, 'contrast': contrast, 'correlation': correlation, 'entropy': entropy, 'inverse difference moments': idf}, ignore_index = True)

<DirEntry 'leaf10_sorbus aucuparia'>
<DirEntry 'leaf11_salix sinerea'>
<DirEntry 'leaf12_populus'>
<DirEntry 'leaf13_tilia'>
<DirEntry 'leaf14_sorbus intermedia'>
<DirEntry 'leaf15_fagus silvatica'>
<DirEntry 'leaf1_ulmus carpinifolia'>
<DirEntry 'leaf2_acer'>
<DirEntry 'leaf3_salix aurita'>
<DirEntry 'leaf4_quercus'>
<DirEntry 'leaf5_alnus incana'>
<DirEntry 'leaf6_betula pubescens'>
<DirEntry 'leaf7_salix alba sericea'>
<DirEntry 'leaf8_populus tremula'>
<DirEntry 'leaf9_ulmus glabra'>
<DirEntry 'leaf_anhui barberry'>
<DirEntry 'leaf_beales barberry'>
<DirEntry 'leaf_big fruited holly'>
<DirEntry 'leaf_camphor tree'>
<DirEntry 'leaf_canadian poplar'>
<DirEntry 'leaf_castor aralia'>
<DirEntry 'leaf_chinese cinnamon'>
<DirEntry 'leaf_chinese horse chestnut'>
<DirEntry 'leaf_chinese redbud'>
<DirEntry 'leaf_chinese toon'>
<DirEntry 'leaf_chinese tulip tree'>
<DirEntry 'leaf_crape myrtle'>
<DirEntry 'leaf_deodar'>
<DirEntry 'leaf_ford woodlotus'>
<DirEntry 'leaf_ginkgo'>
<DirEntry 'leaf_

In [9]:
df

,common name,red_mean,red_var,blue_mean,blue_var,green_mean,green_var,area,perimeter,length,width,aspect ratio,form factor,rectangularity,perimeter ratio of length and width,contrast,correlation,entropy,inverse difference moments
0,sorbus aucuparia,175.413117,89.902019,158.223090,118.536284,170.705951,95.692209,2547445.0,18921.339434,3097.903076,1620.005981,1.912279,0.089415,1.970061,4.010535,75.356344,0.995952,9.438853,0.388849
1,sorbus aucuparia,196.430216,79.807680,178.019189,109.838586,192.480699,81.049172,1634514.5,18703.955534,2757.824219,1387.480957,1.987648,0.058713,2.341018,4.512082,57.308815,0.995926,8.980168,0.429019
2,sorbus aucuparia,190.371075,84.645586,175.592140,112.242956,188.752081,85.362662,1738368.0,21407.024849,2702.096924,1579.382568,1.710856,0.047669,2.454972,4.999913,58.240379,0.996254,8.807776,0.435627
3,sorbus aucuparia,185.160308,83.099103,168.328031,111.148373,183.505444,83.465672,2084601.5,19003.685544,1576.593994,2721.065674,0.579403,0.072537,2.057955,4.421868,51.238502,0.996575,9.040451,0.432259
4,sorbus aucuparia,194.484634,76.452251,172.844500,108.608852,194.585816,74.933301,2282467.5,16497.088441,1693.006958,2685.539795,0.630416,0.105390,1.991983,3.767709,56.014063,0.995524,9.293541,0.442648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027,yew plum pie,238.325835,55.421092,237.629440,57.630225,242.593002,41.559948,155142.5,3169.435093,1484.483887,140.012650,10.602498,0.194078,1.339714,1.951026,9.850830,0.997820,1.460217,0.930480
3028,yew plum pie,235.086392,56.583562,231.701753,65.931880,244.655587,30.072435,208088.0,3749.515389,1751.625732,155.323853,11.277249,0.185997,1.307472,1.966237,10.065629,0.997148,1.880480,0.906011
3029,yew plum pie,241.434539,47.188354,238.358278,57.717060,248.133264,24.358347,141974.5,3270.038124,1522.433594,139.203156,10.936775,0.166846,1.492716,1.967962,6.593927,0.997296,1.357991,0.935189
3030,yew plum pie,243.947001,43.065221,242.001275,50.432570,249.138740,23.235827,114726.5,2826.322395,1344.616089,121.611855,11.056620,0.180480,1.425314,1.927615,5.928906,0.997143,1.145095,0.946968


In [10]:
df.to_csv('features.csv')